In [ ]:
!pip install langchain
!pip install unstructured
!pip install tiktoken
!pip install pinecone-client
!pip install pypdf
!pip install langchain_pinecone
!pip install langchain-community
!pip install PyPDF2
!pip install sentence_transformers
!pip install groq
!pip install langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 710.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 5.2 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import numpy as np
import langchain
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from PyPDF2 import PdfReader
from langchain.document_loaders import PyPDFLoader
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
from langchain_pinecone import PineconeVectorStore
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pathlib
import textwrap
from groq import Groq
from IPython.display import display, Markdown

In [ ]:
GROQ_API_KEY = "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"
PINECONE_API_KEY = "1307be7e-dd9f-41e4-af3d-70166f43ba98"
index_name = "innovent1"

In [ ]:
pc = Pinecone(api_key=PINECONE_API_KEY)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=768,
        metric='cosine',
        pod_type='p1'
    )

index = pc.Index(index_name)

stats = index.describe_index_stats()

print(f"Number of vectors in the index '{index_name}': {stats['total_vector_count']}")

Number of vectors in the index 'innovent1': 981


In [ ]:
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device="cpu")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
query_text = "Knurling operation"

def retrieve_query(query_embedding, k=1):
    matching_results = index.query(vector=query_embedding, top_k=k, include_metadata=True)
    return matching_results
query_embedding = embedding_model.encode(query_text)
query_embedding = query_embedding.tolist() if hasattr(query_embedding, 'tolist') else query_embedding
results = retrieve_query(query_embedding)

print("Retrieved Results:")
for result in results['matches']:
    print(result)

Retrieved Results:
{'id': '364',
 'metadata': {'chunk_id': 364.0,
              'text': 'Fundamentals\t\r'
                      ' \xa0of\t\r'
                      ' \xa0CNC\t\r'
                      ' \xa0Machining\t\r'
                      ' \xa0\t\r'
                      ' \xa0Lesson\t\r'
                      ' \xa06\t\r'
                      ' \xa0CNC\t\r'
                      ' \xa0Operation\t\r'
                      ' \xa0\n'
                      '6-\xad‐18\t\r'
                      ' \xa0\t\r'
                      ' \xa0\t\r'
                      ' \xa0\t\r'
                      ' \xa0Copyright\t\r'
                      ' \xa02014\t\r'
                      ' \xa0Autodesk,\t\r'
                      ' \xa0Inc.\t\r'
                      ' \xa05\t\r'
                      ' \xa0\t\r'
                      ' \xa0Handle\t\r'
                      ' \xa0Jog:\t\r'
                      ' \xa0Select\t\r'
                      ' \xa0Jog\t\r'
                      ' \xa0In

In [ ]:
text = results['matches'][0]['metadata']['text']

In [ ]:
cleaned_text = text.replace('\t', ' ').replace('\r', ' ').replace('\xa0', ' ').replace('§\uf0a7', '').replace('\xad‐', '-')
cleaned_text = ' '.join(cleaned_text.split())

print(cleaned_text)

Fundamentals of CNC Machining Lesson 6 CNC Operation 6--18 Copyright 2014 Autodesk, Inc. 5 Handle Jog: Select Jog Increment: .01 6 Jog Handle: As Needed Select jog direction and use handle as required to place edge finder stylus alongside the left part edge. 7 Jog Increment: .001 Move edge finder slowly until it just trips off center as shown below. This places the center of the spindle exactly .100 from the part edge.


In [ ]:
ai_role = "You are a AI assistant that helps people resolve their error in industrial machinary the user has give the error or problem they are facing and the relevent solution is given convert this entire query into a step by step guide for solving the error"
user_input = query_text
query_to_gen = ai_role + user_input + cleaned_text
query_to_gen

'You are a AI assistant that helps people resolve their error in industrial machinary the user has give the error or problem they are facing and the relevent solution is given convert this entire query into a step by step guide for solving the errorKnurling operationFundamentals of CNC Machining Lesson 6 CNC Operation 6--18 Copyright 2014 Autodesk, Inc. 5 Handle Jog: Select Jog Increment: .01 6 Jog Handle: As Needed Select jog direction and use handle as required to place edge finder stylus alongside the left part edge. 7 Jog Increment: .001 Move edge finder slowly until it just trips off center as shown below. This places the center of the spindle exactly .100 from the part edge.'

In [ ]:
client = Groq(api_key=GROQ_API_KEY)
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": query_to_gen,
        }
    ],
    model="llama3-70b-8192",
)
print(chat_completion.choices[0].message.content)

**Step-by-Step Guide to Resolving Knurling Operation Error**

**Problem Statement:** Error during Knurling operation in CNC Machining.

**Error Description:** The machine is not accurately positioning the edge finder alongside the part edge, resulting in an incorrect knurling operation.

**Solution:**

**Step 1: Handle Jog**

* Select the Jog Increment as `.01`
* Use the Jog Handle as needed to place the edge finder stylus alongside the left part edge.

**Step 2: Jog Direction and Edge Finder Positioning**

* Select the jog direction to move the edge finder slowly towards the part edge.
* Use the Jog Handle to carefully position the edge finder alongside the part edge.

**Step 3: Fine-Tune Edge Finder Position**

* Change the Jog Increment to `.001`
* Move the edge finder slowly until it just trips off center.
* This will place the center of the spindle exactly `.100` from the part edge, ensuring accurate knurling operation.

**Verification:**

* Check the edge finder position to ensur

In [ ]:
import os
from langchain.chains import LLMChain
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_groq import ChatGroq


groq_chat = ChatGroq(
            groq_api_key=GROQ_API_KEY,
            model_name="llama3-70b-8192"
    )

print("Hello! I'm your here to solve any error you face while handling a Machine. I can help answer your questions and solve errors!")

system_prompt = 'You are a friendly conversational chatbot'
conversational_memory_length = 5

memory = ConversationBufferWindowMemory(k=conversational_memory_length, memory_key="chat_history", return_messages=True)
while True:
        user_question = input("Ask a question: (To end chat enter : END) : ")
        if user_question == "END":
          break;
        if user_question:
            user_question = ai_role + user_question + cleaned_text
            prompt = ChatPromptTemplate.from_messages(
                [
                    SystemMessage(
                        content=system_prompt
                    ),

                    MessagesPlaceholder(
                        variable_name="chat_history"
                    ),

                    HumanMessagePromptTemplate.from_template(
                        "{human_input}"
                    ),
                ]
            )
            conversation = LLMChain(
                llm=groq_chat,
                prompt=prompt,
                verbose=False,
                memory=memory,
            )
            response = conversation.predict(human_input=user_question)
            print("Chatbot:", response)

Hello! I'm your friendly Groq chatbot. I can help answer your questions, provide information, or just chat. I'm also super fast! Let's start our conversation!
Ask a question: (To end chat enter : END)END
